In [9]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


ValueError: GPU setup must happen before importing TensorFlow

In [1]:
import numpy as np
import os.path as osp
from evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs

In [2]:
top_sample_dir = '/orions4-zfs/projects/optas/DATA/OUT/iclr/synthetic_samples/gmm/emd/'
class_name = 'chair'
# generative_type = 'latent_gan', 'gmm', or 'raw_gan'
generative_type = 'gmm'
do_jsd = True
do_mmd = False
mmd_loss = 'emd'

In [3]:
if do_jsd:
    f_jsd = open(osp.join(top_sample_dir, 'JSD_Measurements.txt'), 'w')
if do_mmd:
    f_mmd = open(osp.join(top_sample_dir, 'MMD_Measurements.txt'), 'w')

In [4]:
if generative_type == 'latent_gan':
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(100, 2001, 100)])
elif generative_type == 'raw_gan':
    epochs_to_check = np.hstack([np.array([1, 5, 10]), np.arange(50, 2001, 50)])

In [5]:
sample_files = []
if generative_type.endswith('gan'):    
    for epoch in epochs_to_check:
        sample_files.append(osp.join(top_synthetic_dir, 'epoch_%d.npz' % (epoch,) ))
elif generative_type.startswith('gmm'):
    for f in files_in_subdirs(top_sample_dir, '.npz'):
        sample_files.append(f) 
else:
    assert False

In [6]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()
if do_jsd:
    evaluator.prepare_gt_grid_variables()

In [7]:
for sample_file in sample_files:    
    evaluator.prepare_sample_data(sample_file)
    print osp.basename(sample_file)
    if do_jsd:
        print 'JSD'
        f_jsd.write(osp.basename(sample_file) + '\n')
        evaluator.compute_jsd(f_out=f_jsd)
        f_jsd.flush()
    if do_mmd:
        print 'MMD'
        f_mmd.write(osp.basename(sample_file) + '\n')
        evaluator.compute_mmd(loss=mmd_loss, f_out=f_mmd)
        f_mmd.flush()

bneck_128_diag_20_gaussians.npz
JSD


/orions4-zfs/projects/optas/Git_Repos/tf_lab/evaluate/generative_pc_nets.py:39: UserWarning: Point-clouds not in in unit cube.
  warnings.warn('Point-clouds not in in unit cube.')
/orions4-zfs/projects/optas/Git_Repos/tf_lab/evaluate/generative_pc_nets.py:42: UserWarning: Point-clouds not unit sphere.
  warnings.warn('Point-clouds not unit sphere.')


train 0.0212673840574
test 0.0283783834762
val 0.0351918111863
bneck_128_full_40_gaussians.npz
JSD
train 0.00435269430736
test 0.0132044223126
val 0.0187698879158
bneck_128_full_24_gaussians.npz
JSD
train 0.00517805763887
test 0.0132719637427
val 0.0193092456534
bneck_128_full_30_gaussians.npz
JSD
train 0.00468522594561
test 0.0132465520322
val 0.0207734011898
bneck_128_full_18_gaussians.npz
JSD
train 0.00505635290683
test 0.0125019263628
val 0.0220928464384
bneck_128_full_12_gaussians.npz
JSD
train 0.00567643646805
test 0.0140529637012
val 0.0207792792665
bneck_128_full_4_gaussians.npz
JSD
train 0.00888758450095
test 0.0168092111725
val 0.0237142428425
bneck_128_full_10_gaussians.npz
JSD
train 0.00568862543162
test 0.0133000783658
val 0.0190252912663
bneck_128_full_38_gaussians.npz
JSD
train 0.0048442970387
test 0.0136348403225
val 0.0200115531668
bneck_128_full_6_gaussians.npz
JSD
train 0.00800797175244
test 0.0141029873006
val 0.0218804231752
bneck_128_full_26_gaussians.npz
JSD
trai

In [8]:
if do_jsd:
    f_jsd.close()
if do_mmd:
    f_mmd.close()